### SNR-1-Mask

In [1]:
import pandas as pd
from scipy.stats import ttest_ind
import numpy as np
import scipy
from tqdm import tqdm, trange
#from tqdm import tnrange,tqdm_notebook
import matplotlib.pyplot as plt
from scipy.stats import t
import matplotlib.pyplot as plt
import gmpy2
from gmpy2 import mpz,mpq,mpfr,mpc
import os
import scipy.io as sio
import matplotlib.pyplot as plt
from tqdm import tnrange,tqdm_notebook

In [2]:
d_len = 250000
d_file_len = 7713

In [3]:
def concatDataTraces(path):
    mats = []
    index = 0 
    fileNames = np.arange(50, (d_len + 50), 50).tolist()
    for file in os.listdir(path):
        mats.append(sio.loadmat(path+"traces_"+ str(fileNames[index]) +".mat", verify_compressed_data_integrity=False))
        index = index + 1
    mats = np.array(mats)
    myArray = []
    
    for i in range(0,d_file_len):
        for j in range(0,50):
            myArray.append(mats[i]['data'][j])
    return(myArray)

In [4]:
def concatDataLabel(path):
    mats = []
    index = 0 
    fileNames = np.arange(50, (d_len + 50), 50).tolist()
    for file in os.listdir(path):
        mats.append(sio.loadmat(path+"labels_"+ str(fileNames[index]) +".mat", verify_compressed_data_integrity=False))
        index = index + 1
    mats = np.array(mats) 
    myArray = []
    for i in range(0,d_file_len):
        for j in range(0,50):
            myArray.append(mats[i]['data'][0][j])
    return(myArray)

In [5]:
def concatTrigger(path):
    mats = []
    index = 0 
    fileNames = np.arange(50, (d_len + 50), 50).tolist()
    for file in os.listdir(path):
        mats.append(sio.loadmat(path+"trigger_"+ str(fileNames[index]) +".mat", verify_compressed_data_integrity=False))
        index = index + 1
    mats = np.array(mats) 
    myArray = []
    for i in range(0,d_file_len):
        for j in range(0,50):
            myArray.append(mats[i]['data'][0][j])
    return(myArray)

In [6]:
mats = []
mats = sio.loadmat('../real_traces/SBOX_traces_avg_40k/spot_4_12/label/labels_100.mat', verify_compressed_data_integrity=False)

In [7]:
mats['data'][0][1]

148

In [8]:
#plt.plot(mats['data'][0][3][0])
plt.plot(mats['data'][2])

IndexError: index 2 is out of bounds for axis 0 with size 1

In [12]:
mats['data']

array([[208, 148,  18, 179, 218]], dtype=int32)

In [1]:
plt.plot(mats['data'][47][0:5000])

NameError: name 'plt' is not defined

In [2]:
plt.plot(mats['data'][47][323120:500000])

NameError: name 'plt' is not defined

In [3]:
plt.plot(mats['data'][25][5010:323120])

NameError: name 'plt' is not defined

In [ ]:
len(mats['data'][47][5010:323120])

### Load traces

In [ ]:
df_data = concatDataTraces('../real_traces/SBOX_traces_25k_share_1_/spot_4_12/traces/')
df_data = np.array(df_data)

In [ ]:
df_label = concatDataLabel('../real_traces/SBOX_traces_25k_share_1_/spot_4_12/label/')
df_label = np.array(df_label)

In [ ]:
#df_trigger = concatTrigger('../real_traces/SBOX_traces_25k_share_2/spot_4_12/trigger/')
#df_trigger = np.array(df_trigger)

In [ ]:
#len(df_trigger[0])

In [ ]:
#pd.DataFrame(df_trigger)

In [ ]:
#plt.plot(df_trigger)

In [ ]:
df_new_label = []
for i in df_label:
    df_new_label.append([i])

In [ ]:
df_full_data = np.append(df_data, df_new_label, axis=1)

In [ ]:
#last entry (label)
df_full_data[0][-1]

In [ ]:
df_full_data_sort = df_full_data[df_full_data[:,-1].argsort()]

In [ ]:
len(df_full_data_sort[0])

In [ ]:
tmpPv = -1
mainIndex = 0

TMP_ROWS = []
meanRows = []
varRows = []

for i in df_full_data_sort:
    tmpCv = i[-1]
    
    if tmpCv == tmpPv:
        TMP_ROWS.append(i[0:500000])
    else:
        
        if mainIndex != 0:
            
            TMP_ROWS=np.array(TMP_ROWS)
            
            #mean
            tmp = TMP_ROWS.mean(axis=0)
            meanRows.append(tmp)
            
            #var
            tmp2 = TMP_ROWS.var(axis=0)
            varRows.append(tmp2)
            
            TMP_ROWS = []
            TMP_ROWS.append(i[0:500000])
            
        tmpPv = i[-1]
        
    if mainIndex == (len(df_full_data_sort) -1):

        TMP_ROWS=np.array(TMP_ROWS)
        
        #mean
        tmp = TMP_ROWS.mean(axis=0)
        meanRows.append(tmp)
        
        #var
        tmp2 = TMP_ROWS.var(axis=0)
        varRows.append(tmp2)
        
        TMP_ROWS = []
        
    mainIndex = mainIndex + 1

In [ ]:
len(meanRows)

In [ ]:
dev_VAR = []
tmp_prod_VAR = []
tmp_sum_VAR = []

res_MEAN = []
res_VAR = []


for i in tnrange(0,len(varRows[0])):
   
    TMP_ROWS_MEAN = []
    TMP_ROWS_VAR =[]
    
    for j in range(0,256):
        TMP_ROWS_MEAN.append(meanRows[j][i])
        TMP_ROWS_VAR.append(varRows[j][i])
    
    res_MEAN.append(np.array(TMP_ROWS_MEAN).var())
    res_VAR.append(np.array(TMP_ROWS_VAR).mean())

In [ ]:
len(res_VAR)

In [ ]:
SNR = np.array(res_MEAN) / np.array(res_VAR)

In [ ]:
plt.plot(SNR)
plt.axvline(x=5010,color='r')
plt.axvline(x=323120,color='r')
#plt.plot(np.log10(mats['data'][47])-2)

In [ ]:
plt.plot(SNR[5010:323120])

In [ ]:
plt.plot(mats['data'][47][0:5000])